In [32]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pymystem3
import pymorphy2
import nltk
from nltk.stem.snowball import RussianStemmer
import re
from collections import Counter

from tqdm import tqdm_notebook, tqdm
tqdm.pandas(desc="apply progress bar")

from nltk.corpus import stopwords
%matplotlib inline
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from nltk.corpus import brown

import urllib
from bs4 import BeautifulSoup

In [ ]:
with urllib.request.urlopen('http://ezoport.ru/nauki/psixologiya/luiza-xej-tablica-prichin-boleznej-i-zabolevanij-affirmacii-zdorovya.html') as response:
    html = response.read().decode('utf-8')
bs_html = BeautifulSoup(html, "lxml")

full_table = pd.DataFrame({'disease':[], 'fealings': [], 'thinking': []})

disease_list = []
fealings_list = []
thinking_list = []
for i, row in enumerate(bs_html.find_all('tr')):
    try:
        und_rows = row.find_all('td')
        disease = und_rows[0].find_all('strong')[0].text
        
        final_disease = ''
        for word in nltk.word_tokenize(disease):
            if word != '.':
                final_disease += (' ' + word)
        final_disease = final_disease[1:]
        
        feal = und_rows[1].text
        think = und_rows[2].text
        if final_disease != 'ПРОБЛЕМА':
            full_table = full_table.append(pd.DataFrame({'disease': [final_disease],
                                                         'fealings': [feal],
                                                         'thinking': [think]}))
    except:
        # not diseases
        pass
full_table.drop_duplicates('disease', inplace=True)
full_table = full_table.reset_index().drop('index', axis=1)



with urllib.request.urlopen('https://illness.docdoc.ru') as response:
    html = response.read().decode('utf-8')
bs_html = BeautifulSoup(html, "lxml")

link_table = pd.DataFrame({'disease': [], 'link': []})

for row in bs_html.find_all('a'):
    try:
        title = row.attrs['title']
        
        final_title = ''
        for word in nltk.word_tokenize(title):
            if word != '.':
                final_title += (' ' + word)
        final_title = final_title[1:]
        
        link = row.attrs['href']
        if link[:15] != 'https://docdoc.':
            link_table = link_table.append(pd.DataFrame({'disease': [final_title], 'link': [link]}))
    except:
        # not diseases
        pass
link_table.drop_duplicates('disease', inplace=True)
link_table = link_table.reset_index().drop('index', axis=1)

In [ ]:
link_table = link_table.merge(full_table, how='left', left_on='disease', right_on='disease')

list_of_lists = []
for i in tqdm(range(link_table.shape[0])):
    current_s_list = []
    current_link = link_table.iloc[i].link
    with urllib.request.urlopen('https://illness.docdoc.ru' + current_link) as response:
        html = response.read().decode('utf-8')
    bs_html = BeautifulSoup(html, "lxml")
    for sent in bs_html.find_all('ul')[3].find_all('li'):
        current_s_list.append(sent.text)
    list_of_lists.append(current_s_list)

In [ ]:
link_table['symptoms'] = list_of_lists
link_table.drop('link', axis=1, inplace=True)

In [ ]:
punctuation_mine = '!"#$%&\'()*+-./:<=>?@[\\]^_`{|}~'

from tqdm import tqdm_notebook, tqdm
tqdm.pandas(desc="apply progress bar")
from string import punctuation

def clear_symptoms(x):
    clear_text = ''
    current_x = ' '.join(x)
    for sym in current_x:
        if sym not in punctuation_mine:
            clear_text += sym
    return clear_text

link_table['clear_text'] = link_table['symptoms'].progress_apply(clear_symptoms)

In [ ]:
link_table.to_csv('./RoboMed/out.csv', encoding='utf-8')

In [33]:
def add_ngrams(list_of_ends):
        list_of_ngramms = []
        for i in range(len(list_of_ends[-1])):
            list_of_ngramms.append(tuple(list_of_ends[j][i] for j in range(len(list_of_ends))))
        return Counter(list_of_ngramms)

In [34]:
class NGramStorage:
    """Storage for ngrams' frequencies.
    Args:
        sents (list[list[str]]): List of sentences from which ngram
            frequencies are extracted.
        max_n (int): Upper bound of the length of ngrams.
            For instance if max_n = 2, then storage will store
            0, 1, 2-grams.    
    Attributes:
        max_n (Readonly(int)): Upper bound of the length of ngrams.
    """
       
    
    def add_ngrams(self, list_of_ends):
        list_of_ngramms = []
        for i in range(len(list_of_ends[-1])):
            list_of_ngramms.append(tuple(list_of_ends[j][i] for j in range(len(list_of_ends))))
        return Counter(list_of_ngramms)
            
    
    def __init__(self, sents=[], max_n=0):
        self.__max_n = max_n
        self.__ngrams = {i: Counter() for i in range(self.__max_n + 1)}
        
        # self._ngrams[K] should have the following interface:
        # self._ngrams[K][(w_1, ..., w_K)] = number of times w_1, ..., w_K occured in words
        # self._ngrams[0][()] = number of all words
        
        self.__ngrams[0][()] = np.sum([len(sents[i]) for i in range(len(sents))])
        tokens = [' '.join(sents[i]) for i in range(len(sents))]
        tokens = (' '.join(tokens)).split(' ')
        list_of_ends = []
        list_of_ends.append(tokens)
        for n in range(1, self.__max_n + 1):
            self.__ngrams[n] = add_ngrams(list_of_ends)
            list_of_ends.append(tokens[n:])
    
    def ret_ngrams(self):
        return self.__ngrams


    def add_unk_token(self):
        """Add UNK token to 1-grams."""
        # In order to avoid zero probabilites 
        if self.__max_n == 0 or 'UNK' in self.__ngrams[1]:
            return
        self.__ngrams[0][()] += 1
        self.__ngrams[1][('UNK', )] = 1
        
    @property
    def max_n(self):
        """Get max_n"""
        return self.__max_n
        
    def __getitem__(self, k):
        """Get dictionary of k-gram frequencies.
        Args:
            k (int): length of returning ngrams' frequencies.
        Returns:
            Dictionary (in fact Counter) of k-gram frequencies.
        """
        # Cheking the input
        if not isinstance(k, int):
            raise TypeError('k (length of ngrams) must be an integer!')
        if k > self.__max_n:
            raise ValueError('k (length of ngrams) must be less or equal to the maximal length!')
        return self.__ngrams[k]
    
    def __call__(self, ngram):
        """Return frequency of a given ngram.
        Args:
            ngram (tuple): ngram for which frequency should be computed.    
        Returns:
            Frequency (int) of a given ngram.
        """
        # Cheking the input
        if not isinstance(ngram, tuple):
            raise TypeError('ngram must be a tuple!')
        if len(ngram) > self.__max_n:
            raise ValueError('length of ngram must be less or equal to the maximal length!')
        if len(ngram) == 1 and ngram not in self.__ngrams[1]:
            return self.__ngrams[1][('UNK', )]
        return self.__ngrams[len(ngram)][ngram]

In [35]:
analyser = pymorphy2.MorphAnalyzer()
stem = RussianStemmer()
morfer = pymorphy2.MorphAnalyzer()

def do_stem(text):
    new_text = ''
    for token in nltk.word_tokenize(text):
        new_text += stem.stem(token) + ' '
    if len(new_text) > 0:
        new_text = new_text[:-1]
    return new_text

# Ищем прилагательные
def find_ADJF(text):
    list_of_ADJF = []
    text = "делал " + text
    tokens = nltk.word_tokenize(text)
    for i in range(1, len(tokens) - 1):
        current_text = ""
        while analyser.parse(tokens[i])[0].tag.POS == 'ADJF' or\
                (analyser.parse(tokens[i - 1])[0].tag.POS == 'CONJ' and\
                 analyser.parse(tokens[i - 1])[0].tag.POS == 'ADJF'):
            current_text += tokens[i] + " "
            i += 1
        if current_text != "":
            if analyser.parse(tokens[i])[0].tag.POS == 'NOUN':
                current_text += tokens[i]
                list_of_ADJF.append(current_text)
    return list_of_ADJF

# Ищем фразы в текстах
def find_frases(text):
    list_of_frases = []
    tokens = nltk.word_tokenize(text)
    for i in range(1, len(tokens) - 1):
        current_text = ""
        while analyser.parse(tokens[i])[0].tag.POS == 'ADJF':
            current_text += tokens[i] + " "
            i += 1
        if current_text != "":
            if analyser.parse(tokens[i])[0].tag.POS == 'NOUN':
                current_text += tokens[i]
                list_of_ADJF.append(current_text)
    return list_of_ADJF
   

# Приводим к нормальной форме слова в текстах
def do_morfy(text):
    new_text = ''
    for token in nltk.word_tokenize(text):
        new_text += morfer.parse(token)[0].normal_form + ' '
    if len(new_text) > 0:
        new_text = new_text[:-1]
    return new_text

# Приводим к нормальной форме существительные в текстах
def do_morfy_nouns(text):
    new_text = ''
    for token in nltk.word_tokenize(text):
        current_token = morfer.parse(token)[0]
        is_noun = current_token.tag.POS == 'NOUN'
        if is_noun:
            new_text += current_token.normal_form + ' '
        else:
            new_text += token + ' '
    if len(new_text) > 0:
        new_text = new_text[:-1]
    return new_text

# Убираем строки, не относящиеся к рассматриваемой категории
def find_product_cat(text):
    for category_cat in products_dict:
        if text in products_dict[category_cat]:
            return category_cat
    return 'Unknown'

# Для нахождения популярных существительных
def nouns_counter(text, count_rate):
    text = re.sub(' и ', ' ', text)
    words_dict = {}
    text = do_morfy(text)
    nouns_count = Counter()
    tokens = nltk.word_tokenize(text)
    for i in range(len(tokens)):
        if analyser.parse(tokens[i])[0].tag.POS == 'NOUN':
            if tokens[i] not in words_dict:
                words_dict[tokens[i]] = Counter()
            nouns_count[tokens[i]] += count_rate
            i_back = i - 1
            i_up = i + 1
            while i_back >= 0 and analyser.parse(tokens[i_back])[0].tag.POS in ['ADJF', 'ADJS']:
                words_dict[tokens[i]][tokens[i_back]] += count_rate
                i_back -= 1
            while i_up < len(tokens) and analyser.parse(tokens[i_up])[0].tag.POS in ['ADJF', 'ADJS']:
                words_dict[tokens[i]][tokens[i_up]] += count_rate
                i_up += 1
    return words_dict, nouns_count

def fill_global_counter(ADJF_c, noun_c, current_ADJF_c, current_noun_c, category):
    for noun in current_ADJF_c:
        if noun not in ADJF_c[category]:
            ADJF_c[category][noun] = Counter()
        for adj in current_ADJF_c[noun]:
            ADJF_c[category][noun][adj] += current_ADJF_c[noun][adj]
    for noun in current_noun_c:
        noun_c[category][noun] += current_noun_c[noun]
    return ADJF_c, noun_c

In [5]:
text = pd.read_csv('./Base_3.csv', encoding='cp1251', index_col=0, dtype={'Жалобы': str})
text.columns = ['review']
text['len'] = text.review.apply(lambda x: len(str(x)))
text = text[(text['len'] > 120) & (text['len'] < 500)]
text.reset_index(inplace=True)
text.drop('index', axis=1, inplace=True)
text['review'] = text['review'].apply(lambda x: str.lower(str(x)))

In [6]:
# Находим прилагательные-стопслова и приводим в простую форму
ADJF_stop_words = []
for word in stopwords.words('russian'):
    if morfer.parse(word)[0].tag.POS == 'ADJF':
        ADJF_stop_words.append(morfer.parse(word)[0].normal_form)

# Удаляем прилагательные-стопслова из отзывов
def delete_stop_words(text):
    new_text = ''
    for token in nltk.word_tokenize(text):
        if morfer.parse(token)[0].normal_form not in ADJF_stop_words:
            new_text += token + ' '
    return new_text

text['review'] = text['review'].progress_apply(delete_stop_words)

apply progress bar: 100%|██████████| 60016/60016 [09:57<00:00, 113.13it/s]


In [247]:
link_table = pd.read_csv('./final_out.csv', index_col=0)
link_table['clear_symptoms'] = link_table['clear_symptoms'].apply(lambda yy: [x[1:-1] for x in yy[1:-1].split(', ')])
link_table['symptoms'] = link_table['symptoms'].apply(lambda yy: [x[1:-1] for x in yy[1:-1].split(', ')])

In [41]:
link_table = pd.read_csv('./out.csv', index_col=0)
link_table['clear_text'] = link_table['clear_text'].apply(str)
link_table['clear_text'] = link_table['clear_text'].apply(do_morfy)

In [7]:
text['review'] = text['review'].apply(str)
text['review'] = text['review'].progress_apply(do_morfy)

apply progress bar: 100%|██████████| 60016/60016 [09:48<00:00, 102.01it/s]


In [8]:
final_list_of_ngrams = []
for ll in text.review.values:
    final_list_of_ngrams.append(nltk.word_tokenize(ll))
storage = NGramStorage(final_list_of_ngrams, 7)
ngrams_jalobi = storage.ret_ngrams()

final_counter_jalobi = Counter()
for i in tqdm_notebook(range(1,8)):
    fff = ngrams_jalobi[i]
    for obj_i in fff.most_common(1000):
        if (',' in obj_i[0]) or ('.' in obj_i[0]) or ('(' in obj_i[0]) or (')' in obj_i[0])\
               or ('-' in obj_i[0]) or ('\'' in obj_i[0]) or (':' in obj_i[0]) or (')' in obj_i[0]):
            pass
        else:
            current_obj_i = ' '.join(list(obj_i[0]))
            final_counter_jalobi[' '.join(list(obj_i[0]))] += obj_i[1]

A Jupyter Widget

In [51]:
final_list_of_ngrams = []
for ll in link_table.clear_text.values:
    final_list_of_ngrams.append(nltk.word_tokenize(str(ll)))
storage = NGramStorage(final_list_of_ngrams, 7)
ngrams_diseases = storage.ret_ngrams()

final_counter_diseases = Counter()
for i in tqdm_notebook(range(1,8)):
    fff = ngrams_diseases[i]
    for obj_i in fff.most_common(1000):
        if (',' in obj_i[0]) or ('.' in obj_i[0]) or ('(' in obj_i[0]) or (')' in obj_i[0])\
               or ('-' in obj_i[0]) or ('\'' in obj_i[0]) or (':' in obj_i[0]) or (';' in obj_i[0]):
            pass
        else:
            current_obj_i = ' '.join(list(obj_i[0]))
            final_counter_diseases[' '.join(list(obj_i[0]))] += obj_i[1]

A Jupyter Widget

In [232]:
link_table

,disease,fealings,thinking,symptoms,clear_text,gg,clear_symptoms
0,Абсцесс,"Беспокоящие мысли об обидах, пренебрежении и м...",Я даю мыслям свободу. С прошлым покончено. У м...,['резкое повышение температуры (до 40 градусов...,резкий повышение температура до 40 градус ; об...,"[('резкий', 27), ('повышение', 59), ('температ...",[отёчность и боль при абсцесс поверхностный ти...
1,Авитаминоз,NaN,NaN,"['бледная вялая кожа;', 'сухость, раздражение ...","бледный вялый кожа ; сухость , раздражение кож...","[('бледный', 4), ('кожа', 173), ('сухость', 31...",[обострение хронический заболевание рецидив ге...
2,Аденовирус,NaN,NaN,"['симптомы фарингита (боль в горле, воспаление...","симптом фарингит боль в горло , воспаление гор...","[('симптом', 22), ('фарингит', 7), ('боль', 25...","[повышение температура тело до 39 градус, восп..."
3,Аденома гипофиза,NaN,NaN,"['пролактиномы (пролактиновые аденомы);', 'сом...",пролактином пролактиновый аденома ; соматотроп...,"[('аденома', 8), ('аденома', 8), ('аденома', 8...",[аденома]
4,Аденома простаты,NaN,NaN,['Обструктивные:- затруднение в процессе мочеи...,обструктивный затруднение в процесс мочеиспуск...,"[('затруднение', 15), ('в', 766), ('процесс', ...",[обструктивный затруднение в процесс мочеиспус...
5,Аденома щитовидной железы,NaN,NaN,"['возраст пациента старше 40 лет;', 'принадлеж...",возраст пациент старший 40 год ; принадлежност...,"[('возраст', 16), ('пациент', 23), ('старший',...","[формирование нетоксический узел в щитовидка, ..."
6,Аденомиоз,NaN,NaN,['испытывающие постоянные стрессы или физическ...,испытывать постоянный стресс или физический на...,"[('испытывать', 8), ('постоянный', 31), ('стре...",[испытывать постоянный стресс или физический н...
7,Аднексит,NaN,NaN,"['острый аднексит – начальная степень;', 'подо...",острый аднексито – начальный степень ; подостр...,"[('острый', 51), ('–', 172), ('степень', 17), ...",[обычно наблюдаться при микозной или туберкулё...
8,Акромегалия,NaN,NaN,"['увеличение нижней челюсти, скуловых костей, ...","увеличение нижний челюсть , скуловой кость , н...","[('увеличение', 54), ('нижний', 20), ('челюсть...",[изменение прикус появление межзубный промежут...
9,Алалия,NaN,NaN,"['прием медикаментов;', 'физиотерапевтические ...",приём медикамент ; физиотерапевтический процед...,"[('приём', 34), ('медикамент', 5), ('процедура...","[медикамент, процедура, работа, приём]"


In [237]:
link_table[link_table['disease'] == 'Анальная трещина'].symptoms.values

array([ "['сильную боль в области заднего прохода;', 'спазмы сфинктера;', 'кровотечения из заднего прохода (особенно после дефекации);', 'постоянный зуд в области заднего прохода, усиливающийся при ходьбе или после длительного сидения.']"], dtype=object)

In [273]:
a = 1.6

In [270]:
def noraml_elem_of_diseases(x):
    x = do_morfy(str(x))
    list_of_symptoms = []
    tokens = nltk.word_tokenize(x)
    for i in range(1, 8):
        for j in range(len(x) - i):
            if ' '.join(tokens[j: j + i]) in final_counter_diseases:
                list_of_symptoms.append((' '.join(tokens[j: j + i]),
                                         final_counter_diseases[' '.join(tokens[j: j + i])]))
    for_return = sorted(list(np.unique([x[0] for x in list_of_symptoms])), key=lambda x : len(x), reverse=True)
    return for_return[:int(len(for_return) / 2) + 1]


def what_disease(link_table, jaloba_mine):
    max_score = 0
    max_disease = ''
    for i in range(link_table.shape[0]):
        current_disease = link_table.iloc[i].disease
        set_symptoms = set(link_table.iloc[i].clear_symptoms)
        set_jaloba = set(jaloba_mine)
        inters = set_jaloba.intersection(set_symptoms)
        score = len(inters) / len(set_jaloba)
        if (score > max_score) & (len(set_symptoms) > 4):
            max_score = score
            max_disease = current_disease
    return score, max_disease

In [271]:
# jaloba_mine = 'жалуюсь на повышение температуры, общую слабость, отёчность и боль при абсцессе'
jaloba_mine = 'болит живот очень силно тошнит'
jaloba_mine = noraml_elem_of_diseases(jaloba_mine)
score, result = what_disease(link_table, jaloba_mine)
current_tabel = link_table[link_table['disease'] == result]
disease = current_tabel.disease.values[0]
fealings = current_tabel.fealings.values[0]
thinking = current_tabel.thinking.values[0]
symptoms = current_tabel.symptoms.values[0]


text = "Добрый день, я помощник по определению болезни по сырому тексту жалобы.\n"
text += "Введите жалобу в свободном формате (желательно, без ошибок в орфографии)\n"
#process .......
if score == 0.0:
    score = "<0.1"
text += "Ваш ближайший диагноз по описанию: {} с вероятностью {}\n".format(disease, score)
text += "Наблюдается следущая симптоматика:\n"
for i, t in enumerate(symptoms):
    text += str(i + 1) + ') ' + t + '\n'
# text += symptoms
text += '\n'
try:
    np.isnan(fealings)
except:
    text += "Сейчас у вас в голове следущие мысли : "
    text += fealings + '\n'
    text += "Все будет нормально.\nПо дороге к врачу убеждайте себя и повторяйте, что: "
    text += thinking

In [272]:
print(text)

Добрый день, я помощник по определению болезни по сырому тексту жалобы.
Введите жалобу в свободном формате (желательно, без ошибок в орфографии)
Ваш ближайший диагноз по описанию: Целиакия с вероятностью <0.1
Наблюдается следущая симптоматика:
1) потеря веса;
2) зловонный жидкий стул;
3) увеличение живота в объеме;
4) отеки;
5) капризност
6) аздражительность;
7) замедление роста и развития.




In [ ]:
global_noun_ADJF_counter = {'get_cats': {}}
global_noun_counter = {'get_cats': Counter()}

for i in tqdm_notebook(range(text.shape[0])):
    a1, a2 = nouns_counter(text.review[i], 1)
    global_noun_ADJF_counter, global_noun_counter =\
        fill_global_counter(global_noun_ADJF_counter, global_noun_counter, a1, a2, 'get_cats')

A Jupyter Widget

In [177]:
def find_in_text(texts, global_ADJF, global_nouns):
    nouns_dict = {}
    for noun in tqdm_notebook([nou[0] for nou in global_nouns['get_cats'].most_common()[:100]]):
        current_noun_list = []
        current_ADJF = [adj[0] for adj in global_ADJF['get_cats'][noun].most_common()[:20]]
        for ADJF in current_ADJF:
            rating_list = []
            for i in range(len(texts)):
                if re.search(noun + ' ' + ADJF, texts[i]) or re.search(ADJF + ' ' + noun, texts[i]):
                    rating_list.append(1)
            if len(rating_list) > 0:
                current_noun_list.append((ADJF, rating_list))
        if len(current_noun_list) > 0:
            nouns_dict[noun] = current_noun_list
    return nouns_dict

In [178]:
gg = find_in_text(text['review'], global_noun_ADJF_counter, global_noun_counter)

In [203]:
#for normal_text
CneNP = []
neNPC = []
CNP = []
NPC = []
CNGC = []
CNG = []
CneNG = []
PCNP = []
normal_text = text['review']
for j in tqdm_notebook(range(len(normal_text))):
    tokens = nltk.word_tokenize(normal_text[j])
    for i in range(len(tokens) - 3):
        if morfer.parse(tokens[i])[0].tag.POS == 'NOUN' and tokens[i + 1] == 'не' and\
        morfer.parse(tokens[i + 2])[0].tag.POS == 'ADVB' and\
        (morfer.parse(tokens[i + 3])[0].tag.POS == 'ADJF' or morfer.parse(tokens[i + 3])[0].tag.POS == 'ADJS'):
            CneNP.append((morfer.parse(tokens[i])[0].normal_form,
                          tokens[i] + ' ' + tokens[i + 1] + ' ' + tokens[i + 2] + ' ' + tokens[i + 3]))
        elif (morfer.parse(tokens[i])[0].tag.POS == 'ADJF' or morfer.parse(tokens[i])[0].tag.POS == 'ADJS') and\
        morfer.parse(tokens[i + 1])[0].tag.POS == 'NOUN' and\
        morfer.parse(tokens[i + 2])[0].tag.POS == 'ADVB' and\
        (morfer.parse(tokens[i + 3])[0].tag.POS == 'ADJF' or morfer.parse(tokens[i + 3])[0].tag.POS == 'ADJS'):
            if len(tokens[i-1]) > 1 or tokens[i-1] in ['в']:
                PCNP.append((morfer.parse(tokens[i + 1])[0].normal_form,
                            tokens[i-1] + ' ' + tokens[i] + ' ' + tokens[i + 1] + ' ' + tokens[i + 2] + ' ' + tokens[i + 3]))
            else:
                PCNP.append((morfer.parse(tokens[i + 1])[0].normal_form,
                            tokens[i] + ' ' + tokens[i + 1] + ' ' + tokens[i + 2] + ' ' + tokens[i + 3]))
        elif tokens[i] == 'не' and morfer.parse(tokens[i + 1])[0].tag.POS == 'ADVB' and\
        (morfer.parse(tokens[i + 2])[0].tag.POS == 'ADJF' or morfer.parse(tokens[i + 2])[0].tag.POS == 'ADJS') and\
        morfer.parse(tokens[i + 3])[0].tag.POS == 'NOUN':
            neNPC.append((morfer.parse(tokens[i + 3])[0].normal_form,
                          tokens[i] + ' ' + tokens[i + 1] + ' ' + tokens[i + 2] + ' ' + tokens[i + 3]))
        elif morfer.parse(tokens[i])[0].tag.POS == 'NOUN' and\
        morfer.parse(tokens[i + 1])[0].tag.POS == 'ADVB' and\
        (morfer.parse(tokens[i + 2])[0].tag.POS == 'ADJF' or morfer.parse(tokens[i + 2])[0].tag.POS == 'ADJS'):
            CNP.append((morfer.parse(tokens[i])[0].normal_form,
                        tokens[i] + ' ' + tokens[i + 1] + ' ' + tokens[i + 2]))
        elif morfer.parse(tokens[i])[0].tag.POS == 'ADVB' and\
        (morfer.parse(tokens[i + 1])[0].tag.POS == 'ADJF' or morfer.parse(tokens[i + 1])[0].tag.POS == 'ADJS') and\
        morfer.parse(tokens[i + 2])[0].tag.POS == 'NOUN':
            NPC.append((morfer.parse(tokens[i + 2])[0].normal_form,
                        tokens[i] + ' ' + tokens[i + 1] + ' ' + tokens[i + 2]))
        elif morfer.parse(tokens[i])[0].tag.POS == 'NOUN' and\
            morfer.parse(tokens[i + 1])[0].tag.POS == 'ADVB' and\
            (morfer.parse(tokens[i + 2])[0].tag.POS == 'INFN' or morfer.parse(tokens[i + 2])[0].tag.POS == 'VERB') and\
            morfer.parse(tokens[i + 3])[0].tag.POS == 'NOUN':
                CNGC.append((morfer.parse(tokens[i])[0].normal_form,
                             tokens[i] + ' ' + tokens[i + 1] + ' ' + tokens[i + 2] + ' ' + tokens[i + 3]))
        elif morfer.parse(tokens[i])[0].tag.POS == 'NOUN' and\
        morfer.parse(tokens[i + 1])[0].tag.POS == 'ADVB' and\
        (morfer.parse(tokens[i + 2])[0].tag.POS == 'INFN' or morfer.parse(tokens[i + 2])[0].tag.POS == 'VERB'):
            CNG.append((morfer.parse(tokens[i])[0].normal_form,
                        tokens[i] + ' ' + tokens[i + 1] + ' ' + tokens[i + 2]))
        elif morfer.parse(tokens[i])[0].tag.POS == 'NOUN' and tokens[i + 1] == 'не' and\
        morfer.parse(tokens[i + 2])[0].tag.POS == 'ADVB' and\
        (morfer.parse(tokens[i + 3])[0].tag.POS == 'INFN' or morfer.parse(tokens[i + 3])[0].tag.POS == 'VERB'):
            CneNG.append((morfer.parse(tokens[i])[0].normal_form,
                          tokens[i] + ' ' + tokens[i + 1] + ' ' + tokens[i + 2] + ' ' + tokens[i + 3]))